In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import shutil
import subprocess
from omegaconf import OmegaConf
import pydoc
from tqdm.notebook import tqdm
from collections import defaultdict
from PIL import Image

import cv2
import numpy as np

import torch
from torch import nn

%matplotlib inline
from matplotlib import pylab as plt

sys.path.append("..")
import utils.common, utils.vis

In [ ]:
def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)

In [ ]:
project_dir = "/Vol1/dbstore/datasets/k.iskakov/projects/face_expression"

# run_name = "run-20200910_120005-2icqhqix" # siamese_mediapipe_2d
# run_name = "run-20200910_142958-27s0s7pi" # siamese_dropout
# run_name = "run-20200911_003858-1zs8eycn" # siamese_jaw_pose.weight-5.0
# run_name = "run-20200911_005939-vwlb8xjz" # siamese_small

# run_name = "run-20200915_200519-nitmitct" # siamese_bbox_filter

# run_name = "run-20200916_182119-3khcgm0a"  # siamese_normalize_area-False_jaw_pose_weight-10.0
# run_name = "run-20200916_212159-2kfaqcv4" # siamese_keypoint_l2_normalize_area-False
# run_name = "run-20200917_122110-1qk66uzu" # siamese+keypoint_l2_loss+normalize-image_shape

# run_name = "run-20200917_181208-g15oyjuo" # siamese+mediapipe_normalization
# run_name = "run-20200917_181214-2p2mcq7o" # siamese+mediapipe_normalization+expression_weight-10
# run_name = "run-20200917_181220-23sm1rck" # siamese+mediapipe_normalization+use_beta-false

# run_name = "run-20200923_190202-3lf0gggu"  # siamese+keypoints_3d
# run_name = "run-20200923_185641-256g37gk"  # siamese+mouth
run_name = "run-20200923_180309-2vciol9p"  # siamese+keypoints_3d_loss+expression_loss
# run_name = "run-20200923_180225-3kupdul7"  # siamese+keypoints_3d_loss

experiment_dir = os.path.join(project_dir, "wandb", run_name)

# checkpoint
checkpoint_path = utils.common.get_lastest_checkpoint(os.path.join(experiment_dir, "checkpoints"))
checkpoint_name = os.path.basename(checkpoint_path)
print(f"Checkpoint: {os.path.basename(checkpoint_path)}")

# load config
config_path = os.path.join(experiment_dir, "config.yaml")
with open(config_path) as f:
    config = OmegaConf.load(f)

In [ ]:
# runner
runner_cls = pydoc.locate(config.runner.cls)
runner = runner_cls(config)
runner = runner.to(config.device)

state_dict = torch.load(checkpoint_path)
runner.load_state_dict(state_dict)

runner.eval();

In [ ]:
from train import get_dataloaders

# load azure_people_test dataset
azure_people_test_data_config_path = "../config/data/azure_people_test.yaml"
with open(azure_people_test_data_config_path) as f:
    azure_people_test_data_config = OmegaConf.load(f)
    
config.data.test = azure_people_test_data_config.data.test

modes = ('test',)
for mode in modes:
    config.data[mode].dataloader.args.batch_size = 128
    config.data[mode].dataloader.args.num_workers = 8
    config.data[mode].dataloader.args.shuffle = False
    
    
#     config.data[mode].dataset.args.sample_range = [0, float('+inf'), 5]

dataloaders = get_dataloaders(config, splits=modes)
dataloader = dataloaders['test']

In [ ]:
# setup dirs for result
root_result_dir = os.path.join(config.log.project_dir, "artifacts", "azure_people_video")
result_dir = os.path.join(root_result_dir, config.log.experiment_name)

frame_dir = os.path.join(result_dir, f"frames#{config.log.experiment_name}#{checkpoint_name}")
output_video_path = os.path.join(result_dir, f"video#{config.log.experiment_name}#{checkpoint_name}.mp4")
smplx_dir = os.path.join(result_dir, f"smplx#{config.log.experiment_name}#{checkpoint_name}")

shutil.rmtree(output_video_path, ignore_errors=True)
for d in (frame_dir, smplx_dir):
    shutil.rmtree(d, ignore_errors=True)
    os.makedirs(d, exist_ok=True)
    
# copy smplx structure
op_root = "/Vol1/dbstore/datasets/k.iskakov/azure_people_test/openpose_vakhitov_format"
smplx_root = "/Vol1/dbstore/datasets/a.vakhitov/kinect_dataset/full_testcap_30"

copytree(smplx_root, smplx_dir)

In [ ]:
# this dict will be soon edited and then saved
smplx_dict = defaultdict(dict)

subject_ids = sorted(os.listdir(op_root))
for subject_id in tqdm(subject_ids):
    camera_ids = sorted(os.listdir(os.path.join(op_root, subject_id)))
    
    for camera_id in camera_ids:
        expression_path = os.path.join(smplx_dir, subject_id, "mv", camera_id, "joints_op_face_hand", "expressions.npy")
        expression = np.load(expression_path)
        
        pose_path = os.path.join(smplx_dir, subject_id, "mv", camera_id, "joints_op_face_hand", "poses.npy")
        pose = np.load(pose_path)
        
        smplx_dict[subject_id][camera_id] = {
            'expression': expression,
            'pose': pose
        }

In [ ]:
n_batches = float('+inf')
# n_batches = 1
count = 0
for i, input_dict in tqdm(enumerate(dataloader), total=min(n_batches, len(dataloader))):
    with torch.no_grad():
        input_dict = utils.common.dict2device(input_dict, config.device, dtype=torch.float32)
        if i >= n_batches:
            break
        
        output_dict = runner.forward(input_dict)
        
        expression_pred_norm = np.abs(output_dict['expression_pred'].cpu().numpy()).mean()
        expression_norm = np.abs(input_dict['expression'].cpu().numpy()).mean()
        print(f"expression norm: {expression_pred_norm}, {expression_norm}")
        
        input_dict = utils.common.dict2device(input_dict, 'cpu')
        output_dict = utils.common.dict2device(output_dict, 'cpu')

        batch_size = input_dict['pose'].shape[0]
        for batch_index in range(batch_size):
            subject_id, camera_id, _, seq_index = [input_dict['key'][key_index][batch_index] for key_index in range(4)]
            print(subject_id, camera_id)
            seq_index = seq_index.item()

            jaw_pose_pred = output_dict['jaw_pose_pred'][batch_index].detach().cpu().numpy()
            expression_pred = output_dict['expression_pred'][batch_index].detach().cpu().numpy()
            
            smplx_dict[subject_id][camera_id]['pose'][seq_index, 32:35] = jaw_pose_pred
            smplx_dict[subject_id][camera_id]['expression'][seq_index] = expression_pred / 100

In [ ]:
# save edited dict
subject_ids = sorted(os.listdir(op_root))
for subject_id in tqdm(subject_ids):
    camera_ids = sorted(os.listdir(os.path.join(op_root, subject_id)))
    
    for camera_id in camera_ids:
        expression_path = os.path.join(smplx_dir, subject_id, "mv", camera_id, "joints_op_face_hand", "expressions_pred.npy")
        np.save(expression_path, smplx_dict[subject_id][camera_id]['expression'])
        
        pose_path = os.path.join(smplx_dir, subject_id, "mv", camera_id, "joints_op_face_hand", "poses_pred.npy")
        np.save(pose_path, smplx_dict[subject_id][camera_id]['pose'])

---

In [ ]:
# vis (iz govna i palok)
sys.path.append("/Vol0/user/k.iskakov/dev/pykinect")
from pose_dataset_example import *

KINECT_NOSE_INDEX = 27
KINECT_EYE_LEFT_INDEX = 28
KINECT_EAR_LEFT_INDEX = 29
KINECT_EYE_RIGHT_INDEX = 30
KINECT_EAR_RIGHT_INDEX = 31

KINECT_FACE_INDICES = [
    KINECT_NOSE_INDEX, 
    KINECT_EYE_LEFT_INDEX, 
    KINECT_EAR_LEFT_INDEX, 
    KINECT_EYE_RIGHT_INDEX,
    KINECT_EAR_RIGHT_INDEX
]

def project_kinect_joints(kinect_joints, P_cw, im_size):
    n_j = kinect_joints.shape[0]
    j_h = np.concatenate([kinect_joints, np.ones((n_j, 1))], axis=1)
    j_cam_proj = j_h @ P_cw.T
    j_scr = j_cam_proj[:, 0:2] / np.tile(j_cam_proj[:, 2].reshape(-1, 1), (1, 2))
    j_scr_i = (j_scr + 0.5).astype(int)
    j_scr_i[:, 0] = np.clip(j_scr_i[:, 0], 0, im_size[0]-1)
    j_scr_i[:, 1] = np.clip(j_scr_i[:, 1], 0, im_size[1]-1)
    return j_scr_i

def crop_bbox_by_keypoints_2d(keypoints_2d):
    x_min, y_min = np.min(keypoints_2d, axis=0)
    x_max, y_max = np.max(keypoints_2d, axis=0)

    width = x_max - x_min
    height = y_max - y_min

    # size = max(width, height)
    bbox = x_min, y_min, x_min + width, y_min + height
    bbox = utils.common.get_square_bbox(bbox)

    bbox = utils.common.scale_bbox(bbox, 3)
    
    return bbox 

def load_joints_poses_dict(root_folder_bt, pid_lbl, dev_lbl):
    joints_json_path = root_folder_bt + '/' + pid_lbl + '/bt.json'
    with open(joints_json_path) as json_file:
        joints_poses_list = json.load(json_file)

    joints_poses_dict = dict()
    for i in tqdm(range(len(joints_poses_list))):
        try:
            joints_poses, joints_mask = get_kinect_joints(joints_poses_list, i, dev_lbl)
            joints_poses_dict[joints_poses_list[i]['frame_index']] = joints_poses
        except Exception as e:
            print(f"Failed to load joints. Reason: {e}")
            
    return joints_poses_dict


def get_smplx_params(dataset_path, dev_lbl, fit_type, pid_lbl, part_id, fid, vposer_ckpt, prefix=''):
    if part_id >= 0:
        part_path = dataset_path + '/' + pid_lbl + '/mv/'+ dev_lbl + '/' + fit_type + '/part_' + str(part_id) + '/'                         
    else:
        part_path = dataset_path + '/' + pid_lbl + '/mv/'+ dev_lbl + '/' + fit_type + '/'                         
    poses = np.load(part_path + f'/poses{prefix}.npy')
    face_expressions = np.load(part_path + f'/expressions{prefix}.npy')
    betas = np.load(part_path + '/betas.npy')    
    fid_lst = np.load(part_path + '/fid_lst.npy')
    with open(part_path + '/config.json', 'r') as f:
        config = json.load(f)
    #do we use vposer embeddings
    is_vposer = config['is_vposer']
    #gender of a subject
    is_male = config['is_male']    
    #id of a device (used to decode the rigid pose of the device)
    
    #load the device pose
    dev_lst = config['dev_lst']
    dev_id = 0
    while dev_lst[dev_id] != dev_lbl:
        dev_id += 1        
    dev_orient = None
    dev_trans = None
    if dev_id > 0:
        dev_orient = np.load(part_path + '/dev_orient.npy')
        dev_trans = np.load(part_path + '/dev_trans.npy')
        
    fid_id = np.nonzero(fid_lst == int(fid))[0][0]
    pose = poses[fid_id]
    expression = face_expressions[fid_id]
        
    rot = pose[-3:]
    trans = pose[-6:-3]    
    if is_vposer:        
        from human_body_prior.tools.model_loader import load_vposer
        #load the vposer model        
        vposer, _ = load_vposer(vposer_ckpt, vp_model='snapshot')
        vposer.eval()                
        pose_body_vp = torch.tensor(pose[0:32]).reshape(1, -1)        
        #convert from vposer to rotation matrices
        pose_body_mats = vposer.decode(pose_body_vp).reshape(-1, 3, 3).detach().cpu().numpy()
        pose_body = np.zeros(63)
        for i in range(0, pose_body_mats.shape[0]):
            rot_vec, jac = cv2.Rodrigues(pose_body_mats[i])
            pose_body[3*i : 3*i+3] = rot_vec.reshape(-1)        
        pose_jaw = pose[32:35]
        pose_eye = pose[35:41]
        pose_hand = pose[41:-6]        
    else:
        pose_body = pose[0:63]
        pose_jaw = pose[63:66]
        pose_eye = pose[66:72]
        pose_hand = pose[72:-6]        
    
    if dev_orient is not None:
        rot_mat, jac = cv2.Rodrigues(rot.reshape(3, 1))
        dev_mat, jac = cv2.Rodrigues(dev_orient.reshape(3, 1))
        rot_mat = dev_mat @ rot_mat 
        rot, jac = cv2.Rodrigues(rot_mat)
        trans = dev_mat @ trans.reshape(3, 1) + dev_trans.reshape(3, 1)    
    rot_t = to_tensor(rot)
    trans_t = to_tensor(trans)
    pose_body_t = to_tensor(pose_body)
    pose_hand_t = to_tensor(pose_hand)
    pose_jaw_t = to_tensor(pose_jaw)
    pose_eye_t = to_tensor(pose_eye)
    betas_t = to_tensor(betas)
    expr_t = to_tensor(expression * 1e2)
    return rot_t, trans_t, pose_body_t, pose_hand_t, pose_jaw_t, pose_eye_t, betas_t, expr_t, is_male, is_vposer

def init_smplx_inf_model(pk_root, is_male, fit_type, neutral=False):
    #set gender-dependent model paths and load model parameters
    if is_male:
        s2v = np.load(pk_root + '/rob75_val/s2k_m.npy')        
        bm_path = pk_root + '/body_models_1/smplx/SMPLX_MALE.npz'
    else:
        s2v = np.load(pk_root + '/rob75_val/s2k_f.npy')        
        bm_path = pk_root + '/body_models_1/smplx/SMPLX_FEMALE.npz'
        
    if neutral:
        s2v = np.load(pk_root + '/rob75_val/s2k_m.npy')
        bm_path = pk_root + '/body_models_1/smplx/SMPLX_NEUTRAL.npz'
        
    w_add = np.load(pk_root + '/rob75_val/weights.npy')
    w_add = softmax(w_add, axis=1)        
    
    #number of the pca components in the hand poses
    n_pca = 6
    if fit_type == 'joints_op_face_hand':
        n_pca = 12
        
    #init the smplx inference model                
    exp_bm = ExpBodyModel(bm_path, is_hand_pca=True, num_pca_comps=n_pca, s2v=s2v, w_add=w_add)
    return exp_bm

In [ ]:
root_folder_bt = '/Vol0/user/r.bashirov/workdir/git/data/smplx_kinect/test_capture/offline_processor2'
root_folder_rgb = "/Vol0/user/r.bashirov/workdir/git/data/smplx_kinect/test_capture/offline_processor2"

pk_root = '/Vol1/dbstore/datasets/a.vakhitov/projects/pykinect_fresh/pykinect/data/fit_realtime_data/'
vposer_ckpt = '/Vol1/dbstore/datasets/a.vakhitov/projects/pykinect_fresh/smplify-x/smplify-x-data/vposer_v1_0/'

fit_type = 'joints_op_face_hand'

start_index, n_frames, step = 10, 700, 2
frame_ids = np.arange(start_index, start_index + n_frames, step)

fit_dataset_path = smplx_dir

dev_lbl = '000062692912'
pid_lbl = '04_simple'

#load device calibs
mkv_meta_json_path = root_folder_bt + '/' + pid_lbl + '/' + dev_lbl + '/mkv_meta.json'
if not os.path.exists(mkv_meta_json_path):
    print('dev not found')
K_rgb, K_d, T_depth_to_rgb = read_device_calibs(mkv_meta_json_path)

joints_poses_dict = load_joints_poses_dict(root_folder_bt, pid_lbl, dev_lbl)

In [ ]:
count = 0
for fid in tqdm(frame_ids):
#     try:
        #load kinect skeletons
        joints_poses = joints_poses_dict[fid]

        #load rgb image   
        img_path = root_folder_rgb + '/' + pid_lbl + '/' + dev_lbl + '/color_undistorted/' + str(fid).zfill(6) + '.jpg'
        img = cv2.imread(img_path)[:, :, ::-1]

        im_size = (img.shape[1], img.shape[0])
#         plt.imshow(img[:, :, ::-1])
#         plt.show()
        if img is None:
            print('no rgb frame')
        
        cimgs = []
        for prefix in ('_pred', ''):
            #load fitted SMPLX params    
            part_id = -1
            rot_t, trans_t, pose_body_t, pose_hand_t, pose_jaw_t, pose_eye_t, betas_t, expr_t, is_male, is_vposer = \
                get_smplx_params(fit_dataset_path, dev_lbl, fit_type, pid_lbl, part_id, fid, vposer_ckpt, prefix=prefix)

            #initialize the model
            exp_bm = init_smplx_inf_model(pk_root, is_male, fit_type, neutral=False)

            #Initialize the renderer    
            faces_np = exp_bm.f.detach().cpu().numpy()
            main_renderer = init_renderer(faces_np)                
            kinect_kintree = load_kintree_kinect(pk_root)

            #infer SMPLX*
            body = exp_bm(rot_t, pose_body_t, pose_hand_t, pose_jaw_t, pose_eye_t, betas_t, trans_t, expr_t)
            #infer SMPLX
            verts = body.v[0].detach().cpu().numpy()

            #get predicted 3d vertices
            joints_pred_3d = verts[-32:].reshape(-1, 3)

            #render the SMPLX model
            main_renderer.render(verts, np.eye(3), K=K_rgb)
            cimg = main_renderer.get_aligned_camera_image()[:, :, :3]
            
            cimgs.append(cimg)

        # crop face
        P_cw = K_rgb @ T_depth_to_rgb[0:3, 0:4]
        keypoints_2d = project_kinect_joints(joints_poses, P_cw, im_size)

        face_bbox = crop_bbox_by_keypoints_2d(keypoints_2d[KINECT_FACE_INDICES])

        img_croppped = utils.common.crop_image(img, face_bbox)
        cimgs_cropped = [utils.common.crop_image(cimg, face_bbox) for cimg in cimgs]

        
        cobmibned_img = np.concatenate([img_croppped, cimgs_cropped[0], cimgs_cropped[1]], axis=1)
        if count % 50 == 0:
            plt.imshow(cobmibned_img)
            plt.show()
        
        img_path = os.path.join(frame_dir, f"{count:06d}.jpg")
        cv2.imwrite(img_path, cobmibned_img[:, :, ::-1])
        
        count += 1
#     except Exception as e:
#         print(f"(!) Failed for reason: {e}")

In [ ]:
cmd = [
    "ffmpeg",
    "-y",
    "-framerate", "25",
    "-i", os.path.join(frame_dir, "%06d.jpg"),
    "-c:v", "libx264",
    "-vf", "fps=25",
    "-vf", "pad=ceil(iw/2)*2:ceil(ih/2)*2",
    "-pix_fmt", "yuv420p",
    output_video_path
]

result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
if result.returncode:
    raise ValueError(result.stderr.decode("utf-8"))

In [ ]:
print(f"Output video path:\n{output_video_path}")

---